# This is basically the last section of the 2024_Quandela_Remote notebook, our group notebook for solving this challenge. All about bonus challenge is here.

In [2]:
import perceval as pcvl
import numpy as np
import math

from perceval.components.unitary_components import PS, BS, PERM
from perceval.components.port import Port, Encoding
from perceval.utils import Encoding, PostSelect, Matrix
from perceval.components import Unitary

In [1]:
# Define the function to evaluate the performance given a unitary operation matrix. Pretty the same. the only difference is the encoding 

def evaluate_matrix_hybrid(m): # This function don't check M. make sure we input a squre matrix!
    dim = len(m) # figure out what is the dimension of the matrix
    q, r = np.linalg.qr(m) # QR decomposition. given a matrix m, we obtain q which we're sure is unitary. This serves the input of the function

    # The following part is essentially copied from the Quandela's source code
    processor = pcvl.Processor("SLOS", dim)
    processor.add(4, pcvl.BS.H()) # This is because mode 4 & 5 are the target qubit for the CCZ gate
    processor.add(0, Unitary(q))
    processor.add(4, pcvl.BS.H())

    # It seems that there is no simple way to change the encoding to hybrid mode. However, That simply means a new desired truth table. We can use the same default encoding and only change the truth table there
    processor.add_port(0, Port(Encoding.DUAL_RAIL, 'ctrl0'))
    processor.add_port(2, Port(Encoding.DUAL_RAIL, 'ctrl1'))
    processor.add_port(4, Port(Encoding.DUAL_RAIL, 'data'))

    for i in range(6, dim):
        processor.add_herald(i, 0)
    
    # Define the states and true tables for a CCNOT gate (after the Hadamard gates added on the data bit, CCZ gate has become a CCNOT gate and that's what we are going to evaluate)
    states = {
    pcvl.BasicState([1, 0, 1, 0, 1, 0]): "000",
    pcvl.BasicState([1, 0, 0, 1, 1, 0]): "010",
    pcvl.BasicState([0, 1, 1, 0, 1, 0]): "100",
    pcvl.BasicState([0, 1, 0, 1, 1, 0]): "110",
    pcvl.BasicState([1, 0, 1, 0, 0, 1]): "001",
    pcvl.BasicState([1, 0, 0, 1, 0, 1]): "011",
    pcvl.BasicState([0, 1, 1, 0, 0, 1]): "101",
    pcvl.BasicState([0, 1, 0, 1, 0, 1]): "111"
    }

    ca = pcvl.algorithm.Analyzer(processor, states)

    # What is the truth table for the CZ gate in the hybrid mode?
    truth_table = {"000": "001", "010": "010", "100": "100", "110": "111", 
                   "001": "000", "011": "011", "101": "101", "111": "110"}
    ca.compute(expected=truth_table)
    # pcvl.pdisplay(ca)
    return ca.performance, ca.fidelity.real

In [6]:
# Use the same algorithm to optimized the CCZ gate in the hybrid encoding
# This is my first time ever to implement an advanced optimization algorithm to solve a real-world problem. I'm so excited! But please see if there is anything we an improve!
# I chose genetic algorithm. Not sure if this is the best choice

# Genetic Algorithm Components
class GeneticAlgorithm:
    def __init__(self, population_size, matrix_shape, mutation_rate, generations):
        self.population_size = population_size
        self.matrix_shape = matrix_shape
        self.mutation_rate = mutation_rate
        self.generations = generations
        self.population = [np.random.rand(*matrix_shape) for _ in range(population_size)]
    
    def fitness(self, matrix):
        a, b = evaluate_matrix_hybrid(Matrix(matrix))
        return 10 * a - 1e7 * (1 - b)  # Maximizing a, giving a large peanalty for b < 1
    
    def select(self):
        # Simple tournament selection
        selected = []
        for _ in range(self.population_size):
            i, j = np.random.randint(0, self.population_size, 2)
            selected.append(self.population[i] if self.fitness(self.population[i]) > self.fitness(self.population[j]) else self.population[j])
        return selected
    
    def crossover(self, parent1, parent2):
        # Single point crossover
        if np.random.rand() < 0.5:  # Swap parents to ensure diversity
            parent1, parent2 = parent2, parent1
        split = np.random.randint(1, np.prod(self.matrix_shape))
        child1_flat = np.concatenate((parent1.flatten()[:split], parent2.flatten()[split:]))
        child2_flat = np.concatenate((parent2.flatten()[:split], parent1.flatten()[split:]))
        return child1_flat.reshape(self.matrix_shape), child2_flat.reshape(self.matrix_shape)
    
    def mutate(self, matrix):
        # Random mutation
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                if np.random.rand() < self.mutation_rate:
                    matrix[i, j] += np.random.normal(0, 1)
        return matrix
    
    def evolve(self):
        for generation in range(self.generations):
            new_population = []
            selected = self.select()
            for i in range(0, self.population_size, 2):
                parent1, parent2 = selected[i], selected[i+1]
                child1, child2 = self.crossover(parent1, parent2)
                new_population.append(self.mutate(child1))
                new_population.append(self.mutate(child2))
            self.population = new_population
            # Optional: Print best fitness in generation
            best_fitness = max(self.fitness(matrix) for matrix in self.population)
            good_matrix = max(self.population, key=self.fitness)
            p, f = evaluate_matrix_hybrid(Matrix(good_matrix))
            print(f"Generation {generation+1}: Best Fitness = {best_fitness}, Performance = {p}, Fidelity = {f}")
        return self.best_solution()
    
    def best_solution(self):
        best_matrix = max(self.population, key=self.fitness)
        return best_matrix, self.fitness(best_matrix)

In [7]:
# Run the genetic algorithm
population_size = 100
matrix_shape = (9, 9) # 3 herald bits should be able to do this. Let's start here! Subject to change in future
mutation_rate = 0.05
generations = 500

ga = GeneticAlgorithm(population_size, matrix_shape, mutation_rate, generations)
best_matrix, best_fitness = ga.evolve()

Generation 1: Best Fitness = -6603929.889419195, Performance = 0.1915056771254519, Fidelity = 0.33960681955240335
Generation 2: Best Fitness = -6884763.02561965, Performance = 0.3403232130463712, Fidelity = 0.31152335711482193
Generation 3: Best Fitness = -6654815.06475047, Performance = 0.3125109821889119, Fidelity = 0.3345181810139708
Generation 4: Best Fitness = -6643685.220140231, Performance = 0.12678211254564103, Fidelity = 0.33563135120386434
Generation 5: Best Fitness = -6219449.056393835, Performance = 0.07135957600133407, Fidelity = 0.37805502300104055
Generation 6: Best Fitness = -6417346.451875545, Performance = 0.20731993932140422, Fidelity = 0.35826514749250615
Generation 7: Best Fitness = -6391300.752095325, Performance = 0.19661720640860192, Fidelity = 0.36086972817326113
Generation 8: Best Fitness = -6620722.539308954, Performance = 0.06668750837613217, Fidelity = 0.3379276793815962


In [ ]:
# CCZ_hybrid gate is basically defined by the optimized m matrix, which we are still running 

# This is our result from main chellenge. to be replaced by the optimization result 
m = Matrix([[0.509824528533959, 0, 0, 0, 0, 0, 0, 0, 0, 0.860278414296864, 0, 0],
                    [0, 0.509824528533959, 0, 0.321169327626332 + 0.556281593281541j, 0, 0, 0.330393705586394,
                        - 0.165196852793197 - 0.286129342288294j, -0.165196852793197 + 0.286129342288294j, 0, 0, 0],
                    [0, 0, 0.509824528533959, 0, 0, 0, 0, 0, 0, 0, 0.860278414296864, 0],
                    [0, 0, 0, 0.509824528533959, 0, 0.321169327626332 + 0.556281593281541j, -0.165196852793197
                        + 0.286129342288294j, 0.330393705586394, -0.165196852793197 - 0.286129342288294j, 0, 0, 0],
                    [0, 0, 0, 0, 0.509824528533959, 0, 0, 0, 0, 0, 0, 0.860278414296864],
                    [0, 0.321169327626332 + 0.556281593281541j, 0, 0, 0, 0.509824528533959, -0.165196852793197
                        - 0.286129342288294j, -0.165196852793197 + 0.286129342288294j, 0.330393705586394, 0, 0, 0],
                    [0, 0.330393705586394, 0, -0.165196852793197 - 0.286129342288294j, 0, -0.165196852793197
                        + 0.286129342288294j, -0.509824528533959, 0, -0.321169327626332 + 0.556281593281541j, 0, 0, 0],
                    [0, -0.165196852793197 + 0.286129342288294j, 0, 0.330393705586394, 0, -0.165196852793197
                        - 0.286129342288294j, -0.321169327626332 + 0.556281593281541j, -0.509824528533959, 0, 0, 0, 0],
                    [0, -0.165196852793197 - 0.286129342288294j, 0, -0.165196852793197 + 0.286129342288294j, 0,
                        0.330393705586394, 0, -0.321169327626332 + 0.556281593281541j, -0.509824528533959, 0, 0, 0],
                    [0.860278414296864, 0, 0, 0, 0, 0, 0, 0, 0, -0.509824528533959, 0, 0],
                    [0, 0, 0.860278414296864, 0, 0, 0, 0, 0, 0, 0, -0.509824528533959, 0],
                    [0, 0, 0, 0, 0.860278414296864, 0, 0, 0, 0, 0, 0, -0.509824528533959]])

dim = len(m) # figure out what is the dimension of the matrix
q, r = np.linalg.qr(m) # QR decomposition. given a matrix m, we obtain q which we're sure is unitary.

CCZ_hybrid = pcvl.Processor("SLOS", dim)
CCZ_hybrid.add(4, pcvl.BS.H()) # This is because mode 4 & 5 are the target qubit for the CCZ gate
CCZ_hybrid.add(0, Unitary(q))
CCZ_hybrid.add(4, pcvl.BS.H())

# Add ports and heralds
CCZ_hybrid.add_port(0, Port(Encoding.DUAL_RAIL, 'ctrl0'))
CCZ_hybrid.add_port(2, Port(Encoding.DUAL_RAIL, 'ctrl1'))
CCZ_hybrid.add_port(4, Port(Encoding.DUAL_RAIL, 'data'))

for i in range(6, dim):
    CCZ_hybrid.add_herald(i, 0)